# Part 2: Creating your first Experiment in CRIPT 

---
---
If you haven't done part 1 of this tutorial series; do that first as there are some required steps that must be completed before begining this one.


Part 2 of this tutorial series will focus on entering our first experiment and all the associated data into the database. We will use the anionic polymerization of styrene with secBuLi as our example.

<p style="text-align:center;"><img src="https://raw.githubusercontent.com/C-Accel-CRIPT/cript_tutorials/master/supporting_files/pics/P2_polystyrene_synthesis.svg" alt="Polystyrene_synthesis" width="600" style="background-color:white;" />


This example will involve the following steps:
    
1. Create a `Collection node`
2. Create a `Expieriment node`
3. Create or find `Material nodes` <br>
    a. Create styrene <br>
    b. Create secBuLi solution <br>
    c. The remaining materials we will find for the tutorial inventory
4. Create 'Process node`
5. Create 'Data nodes' <br>
    a. 1H NMR <br>
    b. SEC trace
6. Create Polystyrene `Material node`
    
---
By the end of Part 2, your CRIPT data graph will look something like this:

    
<p style="text-align:center;"><img src="https://raw.githubusercontent.com/C-Accel-CRIPT/cript_tutorials/master/supporting_files/pics/P2_node_diagram.svg" alt="Polystyrene_synthesis" style="background-color:white;" />
    

---

We will start with the basics: importing 'CRIPT module', and connecting to the CRIPT database. Time we will show an alternative way to import the cript package. This way allows us to drop the 'C.' on all the CRIPT objects which makes for easier typing, but can make it less clear of where objects are coming from. <span style="color:SteelBlue"> (This should be done at the beginning of every program or jupyter notebook.)</span> 

<u>Action:</u> Replace 'user = ###@###.##' with the email address you used for your `User node` or 'user uid'. Then run the cell. <span style="color:gray">(You can run a cell by either clicking on it and hitting the 'play/run' button in the toolbar or click the cell and hit "shift" + "enter".)</span>

<u>Output:</u> You should see "Import sucessful!", "Connection to database '####' successful.", "Login as "###" was sucessfully." 

In [ ]:
from cript import *
print("Import successful!")

# Connect to database
db_username = "########"
db_password = "########"
db_project = "########"
db_database = "########"
user = "johndoe@cript.edu"  # Put your email or uid here
db = CriptDB(db_username, db_password, db_project, db_database, user)

<span style="color:red">A word of warning:</span> Don't run this block of code twice. You will get an error that you are trying to make a second connection to the database, which is not allowed. If it does happen, just reset the kernal. 

## <u>Collection Node</u>

The first item to create for this example `Collection node`. The `Collection node` is a organizational tool, you can think of it as a 'folder' on your computer. It just a way to group experiments together. 

For example, if you are going to run a series of kinetic experiments on a reaction, you can group them all in a collection. Another example is, you are creating a libary of polyesters, that can all be grouped into a single collection. Or all the experiments that make up a publication can be grouped into a collection. Also note that you can have collections with collections. An example of that may be you find a new catalyst that can polymerize styrene. You could have a parent collection which holds several other collections child collections with each child collection deticated to a different aspect of the new catalyst reaseach (such as kinetics study, monomer scope, diblock polymer synthesis, etc.). 

We will create a collection that will hold all the tutorial examples.

<u>Action:</u> Run the following cell.

<u>Output:</u> Print out of the node. 

---

In [ ]:
collection = Collection("Tutorials")
print(collection)

Now that we create the collection we will want to save it and pick a group to own it. So lets view what groups we are apart of again. To do that, we can use the `db.view()` function again and pass it the generic CRIPT `Group node`. This will return just the groups you are apart of. Which at this point should just be two.

<u>Action:</u> Run the following cell.

<u>Output:</u> You should see two groups in the list. "tutorial_###" which is the node you created in part 1 and "CRIPT_community" the node you just joined in part 1.


In [ ]:
groups = db.view(Group)

We will put this collection into "tutorial_####". To do this, we just need to save the node and pass it the group we want to save it to with it.

<span style="color:SteelBlue">In the background it will save the collection and update the group in the database. </span> 

<u>Action:</u> Change the number in the brackets to select the "tutorial_####" group. Then run the following cell.

<u>Output:</u> "Save of 'Anionic Polymerizations' was successful." and "Update of 'tutorial_john' successful!". We will also see the "uid" of the collection node.


In [ ]:
db.save(collection, groups[0])

---
## <u>Experiment Node</u>

Next, lets make our `Experiment node`. The `Experiment node` is just a grouping of `Material nodes`, `Process nodes` and `Data nodes`. You can think of an experiment as a page in the notebook, except it doesn't hold any of the data it just reference the nodes that do. The reason we break out the `Material nodes`, `Process nodes` and `Data nodes` and don't just group them into the `Experiment node` is because a single `Material` may be referenced in multiple experiments and this allows us to just refenence the `Material node` instead of duplicating the data. Additionally, this seperated data model is more flexable to cases where someone makes a material in one experiment and uses it or studies in a different experiment. There is also huge benfits in terms of search perferomance and reduced memory usage, but that's beyond the scope of this tutorial.

Let's now create our `Experiment node` for our example.

<u>Action:</u> Run the following cell.

<u>Output:</u> Print out of the `Experiment node`.

---

In [ ]:
expt = Experiment("Anionic Polymerization of Styrene with SecBuLi")
print(expt)

We now can save the `Experiment` to the `Collection` that we just created.

<u>Action:</u> Run the following cell.

<u>Output:</u> Print out of the `collection node`.


In [ ]:
db.save(expt, collection)
print(collection)

---

So far through we have created this node graph. 

<p style="text-align:center;"><img src="https://raw.githubusercontent.com/C-Accel-CRIPT/cript_tutorials/master/supporting_files/pics/P2_node_diagram_user_to_expt.svg" alt="Polystyrene_synthesis" style="background-color:white;" />


Now that we have our orgaization nodes setup, its time to work on the core nodes that store the data.


---
---

## <u>Materal Node</u>

The `Material node` contains identity and property data for a chemical. This can be a polymer, a small molecule, or a mixture of multiple components. CRIPT is already prepopulated with the most common small molecules (including some property data). You can of course add a material if it doesn't already exist in the database, or reference an existing material and add new data, or completely copy a whole material and modify it to suit your own requirements. In terms of best practices (for small molecules), using existing materials is always prefered, followed by referencing, and coping as the least prefered method as duplicates of the same material hinders the effectiveness of the database. When it comes to polymers, polymers should aways be treated as a new material as the molecular weight distribution, composition, topolology is never perfectly the same.

### Creating a Material Node

For our experiment we will go through how to enter styrene into the database. Styrene is already in the database, so we will just setup this `Material node` as an example and not save it. We will then just reference the existing styrene that's in the database.

When it comes to creating a `Material node`, we start by using the `help()` to see what data is required and to see what data is optional.

<u>Action:</u> Run the following cell.

<u>Output:</u> You will get a large output of text. Lets digest the important parts of this output:

* The first line is: "__ init __ (self, iden: 'Union[list[Iden], Iden, list[Material], Material]', name: str = None ..."
    * Here we can see that `iden` is the only required parameter as its the only parameter that doesn't have a '= None'.
    * If we look at the description of `iden`, it says to run `help(Iden.__init__)`. This is because `Iden` is a helper node for all a materials idenifiers. 


In [ ]:
help(Material.__init__)


<u>Action:</u> Run the following cell.

<u>Output:</u> You will get a large output of text. Lets digest the important parts of this output:

* The first line is: "__ init __ (self, ref_material=None, name: str = None, names: list = None..."
    * Here there are no required properties. However, best practice is to enter as much data as you can.

In [ ]:
help(Iden.__init__)

---

Using the help from the previous outputs, lets now fill out an `Iden helper node` for styrene. Again, best practice here is to fill out as much as you can as this will significant help other find the material when searching.


<u>Action:</u> Run the following cell.

<u>Output:</u> You will get a printout of the node.


In [ ]:
iden = Iden(
        name="styrene",
        names=["vinylbenzene", "phenylethylene", "ethenylbenzene"],
        chem_formula="C8H8",
        smiles="C=Cc1ccccc1",
        cas="100-42-5",
        pubchem_cid="7501",
        inchi_key="PPBRXRYQALVLMV-UHFFFAOYSA-N"
    )

print(iden)

Now that we have specified the identifiers, we can pass it into the `Material node`. We will also add one of the offical CRIPT keywords, and reccomended storage condition (more information on these later). 

<u>Action:</u> Run the following cell.

<u>Output:</u> You will get a printout of the node. We can notice that the idenifiers have been succesfully added to the node along with the new data.

In [ ]:
mat_styrene = Material(
    iden=iden,
    keywords=["styrene"],
    storage=[
        Cond(key="temp", value=-20 * Unit("degC"))
    ]
)

print(mat_styrene)

#### <u>Keywords</u>

We saw that we can add keywords to the material node, we can view the offically supported keywords with the following code `Material.key_table()`. 
These keys are an additional tool to aid in seaching, so its reccommended to add them when possible. You can also specify a custom key if the offical keys don't cover your materal. To enter a custom key add a "+" plus sign at the front of the key to signify that it's not from the offical list (if you don't add the "+" it will raise an error). Example of a custom key: "+soft_polymer" or "+yellow_polymer".

To see the offical list of keys use the `key_table()` call.

<u>Action:</u> Run the following cell.

<u>Output:</u> A table of offical keys will be printed out with a short description.

In [ ]:
Material.key_table()

#### <u>Conditions</u>

We saw that storage used another one of our helper nodes `Cond`. `Cond` is short for condition and is used any time there is a need to specify something about an enviorment. To be more specific we can investigate the `Cond` and the `Cond` keys.  

<u>Action:</u> Run the following cell.

<u>Output:</u> The `help()` will print out parameters that make up a condition. We can see that both 'key' and 'value' are required. Using the `key_table` function once again we can see a list of offically supported keys along with data type, range, database units, and a short discription.

In [ ]:
help(Cond.__init__)
Cond.key_table()

#### <u>Property</u>

One additional helper node that we will want to use is `Prop`. `Prop` is short for property, in which CRIPT currently supports three subclasses of properties:
* Molecular Properties
* Polymer Properties
* Reaction Properties.

Just like `Cond` lets view are required and optional attributes, and what keywords are accepted. 

<u>Action:</u> Run the following cell.

<u>Output:</u> The `help()` will print out attributes that make up a property. We can see that both 'key' and 'value' are required. Using the `key_table` function once again we can see a list of offically supported keys along with type, range, database units, method, conditions and a short discription.


In [ ]:
help(Prop.__init__)
Prop.key_table()

---

If we come back to styrene, we can add several properties to complete the construction of the 'material node'.

<u>Action:</u> Run the following cell.

<u>Output:</u> You will get a printout of the node.

In [ ]:
mat_styrene.prop = [
    Prop(key="phase", value="liquid"),
    Prop(key="color", value="colorless"),
    Prop(key="molar_mass", value=104.15 * Unit("g/mol"), method="prescribed"),
    Prop(key="density", value=0.906 * Unit("g/ml"), cond=[Cond(key="temp", value=25 * Unit("degC"))]),
    Prop(key="bp", value=145 * Unit("degC"), cond=[Cond(key="pres", value=1 * Unit("atm"))]),
    Prop(key="mp", value=-30 * Unit("degC"), cond=[Cond(key="pres", value=1 * Unit("bar"))])
]

print(mat_styrene)

With that, we have completed the construction of the styrene `Material Node`! That was alot of information to take in. So let me do some wrapping up disscusion on this. First we create material identifers, then we passed then into the `Material Node`, and finished by adding the properties. We broke it into these steps to help communicate the process of creating a `Material Node`, however it can be more sinsincly done in one step, as shown below. The last step is to save this material; but as I mentioned before styrene is already in the database. So we will use that styrene node instead.

In [ ]:
# mat_styrene = Material(
#     iden = Iden(
#         name="styrene",
#         names=["vinylbenzene", "phenylethylene", "ethenylbenzene"],
#         chem_formula="C8H8",
#         smiles="C=Cc1ccccc1",
#         cas="100-42-5",
#         pubchem_cid="7501",
#         inchi_key="PPBRXRYQALVLMV-UHFFFAOYSA-N"
#     ),
#     prop = [
#         Prop("phase", "liquid"),
#         Prop("color", "colorless"),
#         Prop("molar_mass", value=104.15 * Unit("g/mol"), method="prescribed"),
#         Prop("density", value=0.906 * Unit("g/ml"), cond=[Cond("temp", 25 * Unit("degC"))]),
#         Prop("bp", 145 * Unit("degC"), cond=[Cond("pres", 1 * Unit("atm"))]),
#         Prop("mp", -30 * Unit("degC"), cond=[Cond("pres", 1 * Unit("bar"))])
#     ],
#     keywords=["styrene"],
#     storage=[Cond("temp", -20 * Unit("degC"))]
# )

# db.save(mat_styrene, expt)
# print(mat_styrene)

---

### <u>Inventory Node</u>

THe `Inventory Node` is a collection of `Material Nodes`. This can represent a whole labs inventory of chemicals, or just a small list of specific chemicals (for exmple ROP monomers). They are very useful as an orgniational tool, and they to can reduce the time it takes to write up an experiment.  

To demonstrate the utility of the `Inventory Node`, we will use it here to get all the `Material Nodes` we need for this tutorial. An `Inventory` with all the materials for this tutorial was already created and can be found in the 'CRIPT_community'. Since we joined the 'CRIPT_community' we can just use `db.view()` to retrieve the inventory.    

<u>Action:</u> Run the following cell.

<u>Output:</u> The print out attributes should show multiple materials in the 'c_material' section. We will then reference these materials in the `Process Node` section.


In [ ]:
inv_doc = db.view(Inventory)
inv = load(inv_doc[0]) # You may need to change the index if .CRIPT_community is not the first one 
print(inv)

---

### <u>Process Node</u>

The `process node` contains the ingredients, quantities, and procedure information for a given process. A process can be anything from a reaction to a material processing step. 

Just like with other nodes lets view what are the required and optional attributes, and what keywords are accepted. 

<u>Action:</u> Run the following cell.

<u>Output:</u> The `help()` will print out attributes that make up a process. We can see that both 'name', 'ingr', 'procedure' are required. Using the `key_table` function once again we can see a list of offically supported keys along with  a short discription.

In [ ]:
help(Process.__init__)
Process.key_table()

The process for our example, is the anionic polymerization of styrene.

<u>Action:</u> Run the following cell.

<u>Output:</u> Print out of the process node.

In [ ]:
rxn = Process(
    name="Anionic of Styrene",
    ingr=[
        [inv.get("SecBuLi solution 1.4M cHex"), 0.017 * Unit("ml"), "initiator", {"mat_id": "secBuLi"}],
        [inv.get("toluene"), 10 * Unit("ml"), "solvent"],
        [inv.get("styrene"), 0.455 * Unit("g"), "monomer"],
        [inv.get("1BuOH"), 5, "quench", {"eq_mat": "SecBuLi solution"}],
        [inv.get("MeOH"), 100 * Unit("ml"), "workup"]
    ],
    procedure="In an argon filled glovebox, a round bottom flask was filled with 216 ml of dried toluene. The "
              "solution of secBuLi (3 ml, 3.9 mmol) was added next, followed by styrene (22.3 g, 176 mmol) to "
              "initiate the polymerization. The reaction mixture immediately turned orange. After 30 min, "
              "the reaction was quenched with the addition of 3 ml of methanol. The polymer was isolated by "
              "precipitation in methanol 3 times and dried under vacuum.",
    cond=[
        Cond("temp", 25 * Unit("degC")),
        Cond("time", 60 * Unit("min")),
        Cond(key="atm", value=inv.get("Argon"))
    ],
    prop=[
        Prop("yield_mass", 0.47 * Unit("g"), 0.02 * Unit("g"), method="scale")
    ],
    keywords=["polymerization", "living_poly", "anionic", "solution"]
)

print(rxn)

One key thing you might note in the previous out put is there are additional quatnities calculate for some of the ingredients. The CRIPT API has imbeded an auto-calculate tool. When it pass a material into the `Process Node` it will look for properties in the `Material Node` like 'molar_mass', 'density', and 'molar_conc'. With the provided quantity for the ingredient and these other properties, it can complete the remaining calculations for us.

We can also view a nicer output of the ingredients.

<u>Action:</u> Run the following cell.

<u>Output:</u> Table of ingredients.

In [ ]:
print(rxn.ingr)

We can also increase the value of a single ingredient, and the API will re-calcuate everything automatically.

<u>Action:</u> Run the following cell.

<u>Output:</u> Table of ingredients.

In [ ]:
rxn.ingr.scale_one("styrene", 2)
print(rxn.ingr)

Alternatively, we can the re-scale the whole reaction, and once again the API will re-calcuate everything automatically.

<u>Action:</u> Run the following cell.

<u>Output:</u> Table of ingredients.

In [ ]:
rxn.ingr.scale(0.5)
print(rxn.ingr)

The last thing we want to do with the `Process Node` is save it. One nice feature of the API is when we save the node, it will also add all the `Material Nodes` that aren't part of the `Experiment Node` for us.

<u>Action:</u> Run the following cell.

<u>Output:</u> 
* The first line "Save of 'Anionic of Styrene' was successful." says that the `Process Node` was saved.
* The next few lines show what `Material Nodes` were added to the `Experiment Node` since they weren't already present.
* The last text line "Update of 'Anionic Polymerization of Styrene with SecBuLi' successful!" says that the updates to the `Experiment Node` have been save to the database.
* We can also see the printout of the `Experiment node` now has the `Materials` and `Process` added to it.

In [ ]:
db.save(rxn, expt)
print(expt)

---

### <u>Data Node</u>

The `Data Node` contains all the meta-data related to raw or processed data as well as the database link to the actual file. Things that are typically put into a `Data Node` include NMRs, SEC trace, molecular weight distributions, stress-strain curves, etc.

Just like with other nodes lets view what are the required and optional attributes, and what keywords are accepted.

<u>Action:</u> Run the following cell.

<u>Output:</u> The `help()` will print out attributes that make up the `Data Node`. We can see that 'type_' is the only required attributes. Using the `key_table` function once again we can see a list of offically supported keys along with  a short discription.

In [ ]:
help(Data.__init__)
Data.key_table()

For our example here, we will save both a 1H NMR and a SEC trace. The first thing we will need to do is define the file path to those files. The files for the tutorial are included in the CRIPT tutorial repository [here](https://github.com/C-Accel-CRIPT/cript_tutorials/tree/master/supporting_files/tutorial_data).



<u>Action:</u> Download these data files. Modify the paths below to match where you saved the files.

<u>Output:</u> A file path for 1H NMR.

In [ ]:
# file paths format: Path(r"C:\###\###\File.ext")

sec_data_path = Path(r"C:\###\###\Polystyrene_SEC.csv")   
cal_path = Path(r"C:\###\###\sec_calibration_curve.xlsx")
nmr1h_path = Path(r"C:\###\###\Polystyrene_1HNMR.zip")
print(nmr1h_path)

We will start by creating the `Data Node` for the SEC trace first. If we look back up were we ran `Data.key_table` we can see a key for 'sec_trace' which is defined with the x-axis as retention time and the y-axis as signal. We can also include sample preperation steps and SEC conditions into our `Data Node`.

<u>Action:</u> Run the following cell.

<u>Output:</u> A print out of the SEC `Data Node`.

In [ ]:
sec_data = Data(
    name="Crude SEC of polystyrene",
    type_="sec_trace",
    file=File(sec_data_path),
    sample_prep="5 mg of polymer in 1 ml of THF, filtered 0.45um pores.",
    cond=[
        Cond("temp", 30 * Unit("degC")),
        Cond("time", 60 * Unit("min")),
        Cond("solvent", value=inv.get("THF")),
        Cond("+flow_rate", 1 * Unit("ml/min"))
    ],
    calibration=File(cal_path)
)

print(sec_data)

We can do much the same thing for the 1H NMR data.

<u>Action:</u> Run the following cell.

<u>Output:</u> A print out of the 1H NMR `Data Node`.

In [ ]:
nmr_data = Data(
    name="Crude 1H NMR of polystyrene",
    type_="nmr_h1",
    file=File(nmr1h_path),
    cond=[
        Cond("temp", 25 * Unit("degC")),
        Cond("solvent", value=inv.get("CDCl3")),
    ]
)

print(nmr_data)

The last thing we need to do is now save the data nodes to our experiment.

<u>Action:</u> Run the following cell.

<u>Output:</u> A print out of a successful save.

In [ ]:
db.save(sec_data, expt)
db.save(nmr_data, expt)

---

### <u>Material Node: Polymer</u>

The last node we need to create to finish this experimental write up is to create the polystyrene `Material Node`. This is the same process as we did for styrene.

<u>Action:</u> Run the following cell.

<u>Output:</u> A print out of a successful save.

In [ ]:
mat_poly = Material(
    iden=Iden(
        name="polystyrene",
        names=["poly(styrene)", "poly(vinylbenzene)"],
        chem_repeat="C8H8",
        bigsmiles="[H]{[>][<]C(C[>])c1ccccc1[<]}C(C)CC",
        cas="100-42-5"
    ),
    c_process=rxn,
    prop=[
        Prop(key="phase", value="solid"),
        Prop(key="color", value="white"),
        Prop(key="m_n", method="nmr", value=4800 * Unit("g/mol"), uncer=400 * Unit("g/mol"), c_data=nmr_data),
        Prop(key="m_n", method="sec", value=5200 * Unit("g/mol"), uncer=100 * Unit("g/mol"), c_data=sec_data),
        Prop(key="d", method="sec", value=1.03, uncer=0.02, c_data=sec_data)
    ]
)

print(mat_poly)

Now we just save it and we are done!


<u>Action:</u> Run the following cell.

<u>Output:</u> A print out of a successful save.

In [ ]:
db.save(mat_poly, expt)

---
---

## Key Takeaways

* You have seen how now to work with: <br>
    * Collection Node
    * Experiment Node
    * Material Node
    * Process Node
    * Data Node
    * Inventory Node
* You also got exposed to our helper nodes: <br>
    * Condition Helper Node
    * Property Helper Node

---
---

### What's next?

There are still more tutorials focused on specific topics:
* [Advanced_searching](https://github.com/C-Accel-CRIPT/cript_tutorials/blob/master/jupyterlab_tutorials/Advanced_searching.ipynb)
* [Exporting](https://github.com/C-Accel-CRIPT/cript_tutorials/blob/master/jupyterlab_tutorials/Exporting.ipynb)

Or checkout other parts of the [CRIPT tutorial repository](https://github.com/C-Accel-CRIPT/cript_tutorials).